In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
%run "3. Analizar variables dependientes.ipynb"

In [ ]:
def Limpiar_Y_Estructurar_Datos_Para_Modelado(Dataframe, Variable_Dependiente, Variables_Independientes):

    """
    Limpia, estructura y prepara un conjunto de datos para el entrenamiento
    de modelos de machine learning, manejando valores faltantes y filtrando
    observaciones válidas.
    
    Esta función realiza el preprocesamiento esencial de datos que incluye:
    eliminación de observaciones sin variable dependiente, imputación 
    inteligente de valores faltantes en variables independientes usando
    mediana para numéricas y moda para categóricas, y validación final
    de la integridad de los datos preparados.
    
    Parámetros:
    -----------
    Dataframe : pandas.DataFrame
        DataFrame completo que contiene todas las variables del estudio.
        Puede contener valores faltantes que serán procesados.
    
    Variable_Dependiente : str
        Nombre de la columna que representa la variable dependiente (Y).
        Las observaciones sin valor en esta columna serán eliminadas.
    
    Variables_Independientes : list of str
        Lista con los nombres de las columnas que serán utilizadas como
        variables independientes (X) en el modelo.
    
    Retorna:
    --------
    tuple (pandas.DataFrame, pandas.Series)
        - Variables_X: DataFrame con las variables independientes limpias
          y sin valores faltantes, listo para modelado.
        - Variable_Y: Serie con la variable dependiente correspondiente
          a las mismas observaciones de Variables_X.
    
    Ejemplo de uso:
    ---------------
    >>> import pandas as pd
    >>> datos = pd.DataFrame({
    ...     'Edad': [25, None, 35, 40],
    ...     'Salario': [30000, 45000, None, 60000],
    ...     'Satisfaccion': [7, 8, None, 9]
    ... })
    >>> X, y = Limpiar_Y_Estructurar_Datos_Para_Modelado(
    ...     Datos, 'Satisfaccion', ['Edad', 'Salario']
    ... )
    >>> print(len(X))  # 2 (Observaciones válidas)
    
    Notas:
    ------
    - La función imprime un reporte detallado del proceso de limpieza
      incluyendo número de observaciones y tratamiento de faltantes.
    - Variables numéricas: imputación con mediana (robusta a outliers).
    - Variables categóricas: imputación con moda (valor más frecuente).
    - Si persisten valores faltantes después de imputación, se eliminan
      esas observaciones para garantizar datos completos.
    - Los índices entre Variables_X e Variable_Y están alineados.
    
    """
    
    # Filtrar observaciones que tienen la variable dependiente.
    Datos_Modelo = Dataframe.dropna(subset=[Variable_Dependiente]).copy()
    
    print(f"Variable dependiente: {Variable_Dependiente}")
    print(f"Observaciones disponibles: {len(Datos_Modelo)}")
    
    # Preparar variables independientes.
    Variables_X = Datos_Modelo[Variables_Independientes].copy()
    Variable_Y = Datos_Modelo[Variable_Dependiente].copy()
    
    # Imputar valores faltantes en variables independientes.
    print(f"Valores faltantes en X antes de imputación:")
    Faltantes_Antes = Variables_X.isnull().sum()
    Faltantes_Antes = Faltantes_Antes[Faltantes_Antes > 0]
    
    if len(Faltantes_Antes) > 0:
        for Variable in Faltantes_Antes.index:
            if Variables_X[Variable].dtype in ['int64', 'float64']:
                # Variables numéricas: imputar con mediana.
                Variables_X[Variable] = Variables_X[Variable].fillna(Variables_X[Variable].median())
                print(f"  {Variable}: {Faltantes_Antes[Variable]} → imputados con mediana")
            else:
                # Variables categóricas: imputar con moda.
                Moda = Variables_X[Variable].mode()
                if len(Moda) > 0:
                    Variables_X[Variable] = Variables_X[Variable].fillna(Moda[0])
                    print(f"  {Variable}: {Faltantes_Antes[Variable]} → imputados con moda")
    else:
        print("  ✅ No hay valores faltantes")
    
    # Verificar que no queden faltantes.
    Faltantes_Despues = Variables_X.isnull().sum().sum()
    if Faltantes_Despues > 0:
        print(f"⚠️  Quedan {Faltantes_Despues} valores faltantes")
        # Eliminar observaciones con faltantes restantes.
        Indices_Completos = Variables_X.dropna().index
        Variables_X = Variables_X.loc[Indices_Completos]
        Variable_Y = Variable_Y.loc[Indices_Completos]
        print(f"Observaciones finales: {len(Variables_X)}")
    
    return Variables_X, Variable_Y